In [1]:
import random
import torch
import torch.nn as nn
from collections import defaultdict

# === Configuration ===
SECTIONS = ["IVA", "IVB", "IVC", "IVD", "IVE"]
BATCHES = [f"{sec}-A1" for sec in SECTIONS] + [f"{sec}-A2" for sec in SECTIONS]
DAYS = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
TIME_SLOTS = ["8:00 AM", "9:00 AM", "10:15 AM", "11:15 AM", "1:30 PM", "2:45 PM", "3:45 PM", "4:45 PM"]
BREAKS = ["10:00 AM", "12:15 PM", "2:30 PM"]

SUBJECTS_INFO = {
    "OOP Lab": {"hours": 3, "type": "lab", "batchwise": True},
    "EDA Lab": {"hours": 3, "type": "lab", "batchwise": True},
    "POCD Tutorial": {"hours": 2, "type": "tutorial", "batchwise": True},
    "CN1 Tutorial": {"hours": 2, "type": "tutorial", "batchwise": True},
    "P&S Tutorial": {"hours": 2, "type": "tutorial", "batchwise": True},
    "OSPP Lab": {"hours": 2, "type": "lab", "batchwise": True},
}

LAB_ROOMS = [f"LAB{i}" for i in range(1, 6)]

TEACHERS = {
    "OOP Lab": ["Mr. Manjunath Gonal", "Ms. Vijayalakshmi Sajjanar", "Mr. K M M. Rajashekharaiah"],
    "EDA Lab": ["Dr. Sujatha C", "Dr. P. G. Sunitha Hiremath", "Ms. Neha Tarannum", "Dr. Padmashree Desai"],
    "POCD Tutorial": ["Ms. Nirmala Patil", "Dr. Jayalaxmi G.N.", "Ms. Indira Bidari", "Dr. Karibasappa K.G", "Ms. Nagaratna V Yaligar"],
    "CN1 Tutorial": ["Dr. Vijayalakshmi M.", "Ms. Preeti Y R", "Mr. Parikshit Hegade"],
    "P&S Tutorial": ["Dr. Sumedha Shinde", "Dr. G.N. Bhadri", "Dr. D.A. Patil", "Ms. Namrata K."],
    "OSPP Lab": ["Dr. Manohar Madgi", "Dr. Shantala Giraddi", "Dr. Shrinivas D. Desai", "Dr. G S Hanchinamani", "Mr. Prakash Hegade"],
}

PAIRED_SUBJECTS = [
    ("OOP Lab", "EDA Lab"),
    ("POCD Tutorial", "CN1 Tutorial"),
    ("P&S Tutorial", "OSPP Lab"),
]
def is_valid_continuous_block(start_idx, block_size, time_slots):
    if start_idx + block_size > len(time_slots):
        return False
    block = time_slots[start_idx:start_idx + block_size]
    for time in block:
        if time in ["12:15 PM", "1:30 PM"]:
            return False
    return True

def find_slot(duration, day, used_indices):
    for start in range(len(TIME_SLOTS) - duration + 1):
        if not is_valid_continuous_block(start, duration, TIME_SLOTS):
            continue
        indices = list(range(start, start + duration))
        if all(i not in used_indices for i in indices):
            return indices
    return None
batch_schedule = {batch: {day: [None] * len(TIME_SLOTS) for day in DAYS} for batch in BATCHES}
lab_room_occupancy = {room: {day: [None] * len(TIME_SLOTS) for day in DAYS} for room in LAB_ROOMS}
teacher_occupancy = defaultdict(lambda: {day: [None] * len(TIME_SLOTS) for day in DAYS})

# === LSTM Encoding Helpers ===
all_teachers = list({t for sub in TEACHERS.values() for t in sub})
teacher2idx = {t: i for i, t in enumerate(all_teachers)}
idx2teacher = {i: t for t, i in teacher2idx.items()}

subject2idx = {s: i for i, s in enumerate(SUBJECTS_INFO.keys())}
section2idx = {s: i for i, s in enumerate(SECTIONS)}

class TimetableLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, 64, batch_first=True)
        self.fc = nn.Linear(64, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        _, (hn, _) = self.lstm(x)
        out = self.fc(hn[-1])
        return out

# Prepare training data for teacher prediction
train_X, train_Y = [], []
def add_training_sample(section, subject, teacher):
    sec_idx = section2idx[section]
    sub_idx = subject2idx[subject]
    input_seq = torch.tensor([[sec_idx, sub_idx]])
    label = torch.tensor([teacher2idx[teacher]])
    train_X.append(input_seq)
    train_Y.append(label)

def train_lstm_model():
    if not train_X:
        return None
    model = TimetableLSTM(input_dim=max(len(section2idx), len(subject2idx))+1, hidden_dim=16, output_dim=len(teacher2idx))
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    for epoch in range(100):
        total_loss = 0
        for x, y in zip(train_X, train_Y):
            out = model(x)
            loss = criterion(out, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

             # Print epoch number and loss
        print(f"Epoch {epoch+1}/100 - Loss: {total_loss:.4f}")

    return model

def find_slot(duration, day, used_indices):
    for start in range(len(TIME_SLOTS) - duration + 1):
        indices = list(range(start, start + duration))
        hours = [TIME_SLOTS[i] for i in indices]
        if any(h in BREAKS for h in hours):
            continue
        if all(i not in used_indices for i in indices):
            return indices
    return None

# === Schedule Labs/Tutorials First ===
def schedule_batchwise_pairs():
    for subj1, subj2 in PAIRED_SUBJECTS:
        duration = SUBJECTS_INFO[subj1]["hours"]
        for section in SECTIONS:
            a1, a2 = f"{section}-A1", f"{section}-A2"
            assigned_first, assigned_second = False, False
            for _ in range(50):
                days_shuffled = DAYS[:]
                random.shuffle(days_shuffled)
                for day in days_shuffled:
                    indices = find_slot(duration, day, set())
                    if not indices:
                        continue
                    rooms_shuffled = LAB_ROOMS[:]
                    random.shuffle(rooms_shuffled)
                    for room in rooms_shuffled:
                        teacher1 = random.choice(TEACHERS[subj1])
                        teacher2 = random.choice(TEACHERS[subj2])
                        # A1=subj1, A2=subj2
                        if not assigned_first and all(
                            batch_schedule[a1][day][i] is None and
                            batch_schedule[a2][day][i] is None and
                            lab_room_occupancy[room][day][i] is None and
                            teacher_occupancy[teacher1][day][i] is None and
                            teacher_occupancy[teacher2][day][i] is None
                            for i in indices):
                            for i in indices:
                                batch_schedule[a1][day][i] = (subj1, teacher1, room)
                                batch_schedule[a2][day][i] = (subj2, teacher2, room)
                                lab_room_occupancy[room][day][i] = f"{subj1}/{subj2}"
                                teacher_occupancy[teacher1][day][i] = subj1
                                teacher_occupancy[teacher2][day][i] = subj2
                            assigned_first = True
                            add_training_sample(section, subj1, teacher1)
                            add_training_sample(section, subj2, teacher2)
                            break
                        # A1=subj2, A2=subj1
                        if not assigned_second and all(
                            batch_schedule[a1][day][i] is None and
                            batch_schedule[a2][day][i] is None and
                            lab_room_occupancy[room][day][i] is None and
                            teacher_occupancy[teacher2][day][i] is None and
                            teacher_occupancy[teacher1][day][i] is None
                            for i in indices):
                            for i in indices:
                                batch_schedule[a1][day][i] = (subj2, teacher2, room)
                                batch_schedule[a2][day][i] = (subj1, teacher1, room)
                                lab_room_occupancy[room][day][i] = f"{subj2}/{subj1}"
                                teacher_occupancy[teacher2][day][i] = subj2
                                teacher_occupancy[teacher1][day][i] = subj1
                            assigned_second = True
                            add_training_sample(section, subj2, teacher2)
                            add_training_sample(section, subj1, teacher1)
                            break
                    if assigned_first and assigned_second:
                        break
                if assigned_first and assigned_second:
                    break

# === Theory Subject Scheduling ===
THEORY_SUBJECTS = {
    "OOP": {"hours": 3, "related_lab": "OOP Lab"},
    "EDA": {"hours": 3, "related_lab": "EDA Lab"},
    "POCD": {"hours": 2, "related_tutorial": "POCD Tutorial"},
    "CN1": {"hours": 3, "related_tutorial": "CN1 Tutorial"},
    "P&S": {"hours": 2, "related_tutorial": "P&S Tutorial"},
    "OSPP": {"hours": 3, "related_lab": "OSPP Lab"},
}

CLASSROOMS = [f"Room-{i}" for i in range(101, 111)]
section_schedule = {sec: {day: [None] * len(TIME_SLOTS) for day in DAYS} for sec in SECTIONS}
classroom_occupancy = {room: {day: [None] * len(TIME_SLOTS) for day in DAYS} for room in CLASSROOMS}

def schedule_theory_classes(lstm_model):
    for subject, info in THEORY_SUBJECTS.items():
        hours_needed = info["hours"]
        related = info.get("related_lab") or info.get("related_tutorial")
        for section in SECTIONS:
            teacher = None
            if lstm_model:
                try:
                    input_seq = torch.tensor([[section2idx[section], subject2idx[related]]])
                    with torch.no_grad():
                        out = lstm_model(input_seq)
                        pred = torch.argmax(out, dim=1).item()
                        teacher = idx2teacher[pred]
                except:
                    pass
            if not teacher:
                teacher = random.choice(TEACHERS.get(related, []))

            hours_assigned = 0
            for day in DAYS:
                for slot in range(len(TIME_SLOTS)):
                    if TIME_SLOTS[slot] in BREAKS:
                        continue
                    if section_schedule[section][day][slot]:
                        continue
                    if teacher_occupancy[teacher][day][slot]:
                        continue
                    if any(batch_schedule[f"{section}-{b}"][day][slot] for b in ["A1", "A2"]):
                        continue
                    for room in CLASSROOMS:
                        if classroom_occupancy[room][day][slot] is None:
                            section_schedule[section][day][slot] = (subject, teacher, room)
                            teacher_occupancy[teacher][day][slot] = subject
                            classroom_occupancy[room][day][slot] = f"{subject} - {section}"
                            hours_assigned += 1
                            break
                    if hours_assigned >= hours_needed:
                        break
                if hours_assigned >= hours_needed:
                    break

# === Run Everything ===
schedule_batchwise_pairs()
lstm_model = train_lstm_model()
schedule_theory_classes(lstm_model)


def print_batch_timetable(batch_name):
    print(f"\n📘 {batch_name} - Weekly Lab/Tutorial Timetable\n")
    for day in DAYS:
        print(f"{day}:")
        for i, slot in enumerate(TIME_SLOTS):
            entry = batch_schedule[batch_name][day][i]
            if entry:
                subject, teacher, room = entry
                print(f"  {slot:8} : {subject} - {room} ({teacher})")
            else:
                print(f"  {slot:8} : -")
        print()

def print_section_timetable(section_name):
    print(f"\n📗 {section_name} - Weekly Theory Timetable\n")
    for day in DAYS:
        print(f"{day}:")
        for i, slot in enumerate(TIME_SLOTS):
            entry = section_schedule[section_name][day][i]
            if entry:
                subject, teacher, room = entry
                print(f"  {slot:8} : {subject} - {room} ({teacher})")
            else:
                print(f"  {slot:8} : -")
        print()

# === Print All Timetables ===
for batch in BATCHES:
    print_batch_timetable(batch)

for section in SECTIONS:
    print_section_timetable(section)


Epoch 1/100 - Loss: 185.8264
Epoch 2/100 - Loss: 133.3979
Epoch 3/100 - Loss: 81.2338
Epoch 4/100 - Loss: 54.2050
Epoch 5/100 - Loss: 48.2749
Epoch 6/100 - Loss: 46.0180
Epoch 7/100 - Loss: 45.3502
Epoch 8/100 - Loss: 43.8310
Epoch 9/100 - Loss: 44.3573
Epoch 10/100 - Loss: 42.8465
Epoch 11/100 - Loss: 42.0953
Epoch 12/100 - Loss: 39.9977
Epoch 13/100 - Loss: 39.5597
Epoch 14/100 - Loss: 38.6111
Epoch 15/100 - Loss: 38.6670
Epoch 16/100 - Loss: 38.1211
Epoch 17/100 - Loss: 38.0801
Epoch 18/100 - Loss: 37.5835
Epoch 19/100 - Loss: 37.4636
Epoch 20/100 - Loss: 37.1727
Epoch 21/100 - Loss: 37.0595
Epoch 22/100 - Loss: 36.8538
Epoch 23/100 - Loss: 36.7990
Epoch 24/100 - Loss: 36.5117
Epoch 25/100 - Loss: 36.5380
Epoch 26/100 - Loss: 36.3191
Epoch 27/100 - Loss: 36.2863
Epoch 28/100 - Loss: 36.1175
Epoch 29/100 - Loss: 36.0701
Epoch 30/100 - Loss: 35.9001
Epoch 31/100 - Loss: 35.9017
Epoch 32/100 - Loss: 35.7323
Epoch 33/100 - Loss: 35.7357
Epoch 34/100 - Loss: 35.5853
Epoch 35/100 - Loss: 